In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from collections import Counter
import holidays
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output


In [3]:
#### read files
df_train = pd.read_csv("/Users/s0c02nj/Desktop/WNS2/train_NA17Sgz/train.csv")
df_log = pd.read_csv("/Users/s0c02nj/Desktop/WNS2/train_NA17Sgz/view_log.csv")
df_item = pd.read_csv("/Users/s0c02nj/Desktop/WNS2/train_NA17Sgz/item_data.csv")

In [4]:
df_test = pd.read_csv("/Users/s0c02nj/Desktop/WNS2/test.csv")

In [5]:
df_train.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


In [6]:
df_log.head()

,server_time,device_type,session_id,user_id,item_id
0,2018-10-15 08:58:00,android,112333,4557,32970
1,2018-10-15 08:58:00,android,503590,74788,7640
2,2018-10-15 08:58:00,android,573960,23628,128855
3,2018-10-15 08:58:00,android,121691,2430,12774
4,2018-10-15 08:58:00,android,218564,19227,28296


In [7]:
df_item.head()

,item_id,item_price,category_1,category_2,category_3,product_type
0,26880,4602,11,35,20,3040
1,54939,3513,12,57,85,6822
2,40383,825,17,8,279,1619
3,8777,2355,13,58,189,5264
4,113705,1267,17,39,151,10239


In [8]:
df_log_item = pd.merge(left = df_log, 
                  right= df_item, 
                  how='left',
                  left_on=['item_id'], 
                  right_on=['item_id'])

In [9]:
df_log_item.head()

,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type
0,2018-10-15 08:58:00,android,112333,4557,32970,54685.0,16.0,56.0,253.0,3184.0
1,2018-10-15 08:58:00,android,503590,74788,7640,1376.0,7.0,71.0,228.0,545.0
2,2018-10-15 08:58:00,android,573960,23628,128855,4544.0,4.0,38.0,62.0,5609.0
3,2018-10-15 08:58:00,android,121691,2430,12774,904.0,17.0,39.0,252.0,2740.0
4,2018-10-15 08:58:00,android,218564,19227,28296,2304.0,12.0,57.0,54.0,7422.0


In [10]:
df_log_item.nunique()

server_time       82840
device_type           3
session_id      1014970
user_id           89157
item_id          126708
item_price        13207
category_1           17
category_2           79
category_3          335
product_type       7846
dtype: int64

In [11]:
cat_cols = ['device_type', 'category_1', 'category_2', 'category_3', 'product_type', 'item_id']

for col in cat_cols:
    le = preprocessing.LabelEncoder()
    df_log_item[col] = le.fit_transform(df_log_item[col].astype(str))

In [12]:
df_log_item.nunique()

server_time       82840
device_type           3
session_id      1014970
user_id           89157
item_id          126708
item_price        13207
category_1           18
category_2           80
category_3          336
product_type       7847
dtype: int64

In [13]:
df_log_item["cat1_cat2"] = df_log_item["category_1"].astype("str") + df_log_item["category_2"].astype("str")
df_log_item["cat1_cat3"] = df_log_item["category_1"].astype("str") + df_log_item["category_3"].astype("str")
df_log_item["cat2_cat3"] = df_log_item["category_2"].astype("str") + df_log_item["category_3"].astype("str")
df_log_item["cat1_cat2_cat3"] = df_log_item["category_1"].astype("str") + df_log_item["category_2"].astype("str") + df_log_item["category_3"].astype("str")



In [14]:
#us_holidays = holidays.UnitedStates()

In [15]:
##### server feature week, month, day and hour
df_log_item['server_time'] = pd.to_datetime(df_log_item['server_time'])

df_log_item['server_wk'] = df_log_item['server_time'].dt.week
df_log_item['server_mth'] = df_log_item['server_time'].dt.month
df_log_item['server_day'] = df_log_item['server_time'].dt.day
df_log_item['server_hr'] = df_log_item['server_time'].dt.hour
df_log_item['server_weekend'] = (df_log_item['server_time'].dt.weekday >= 5).astype(int)

In [16]:
### impute price
df_log_item['item_price'] = df_log_item['item_price'].apply(lambda x:np.log1p(x))
price_med = df_log_item['item_price'].median()
df_log_item['item_price'] = df_log_item['item_price'].replace(np.nan, price_med, regex = True)

In [17]:
def check_nas(df):
    print(df.isna().sum())

In [18]:
def q3(x):
    return np.percentile(x , 75)

def q1(x):
    return np.percentile(x , 25)

def mode(x):
    return Counter(x).most_common(1)[0][0]

def mode_1(x):
    try:
        return Counter(x).most_common(2)[1][0]
    except:
        return 999999

def mode_2(x):    
    try:
        return Counter(x).most_common(3)[2][0]
    except:
        return 999999
    
# def mode_3(x):    
#     try:
#         return Counter(x).most_common(4)[3][0]
#     except:
#         return 999999

# def mode_4(x):    
#     try:
#         return Counter(x).most_common(5)[4][0]
#     except:
#         return 999999

In [19]:
x = [1,1,1,1,1,1,1,2,2,2,2,3]

In [20]:
mode_1(x)

2

In [21]:
is_click = df_train["is_click"]
df_train = df_train.drop(["is_click"], axis=1)

In [22]:
df_combined = pd.concat([df_train, df_test])

In [23]:
df_log_item[df_log_item['user_id'] == 0].head(8)

,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type,cat1_cat2,cat1_cat3,cat2_cat3,cat1_cat2_cat3,server_wk,server_mth,server_day,server_hr,server_weekend
163910,2018-10-18 14:27:00,0,268465,0,67843,8.236685,6,22,107,5651,622,6107,22107,622107,42,10,18,14,0
163921,2018-10-18 14:27:00,0,268465,0,67843,8.236685,6,22,107,5651,622,6107,22107,622107,42,10,18,14,0
164584,2018-10-18 14:43:00,0,268465,0,67843,8.236685,6,22,107,5651,622,6107,22107,622107,42,10,18,14,0
164700,2018-10-18 14:46:00,0,268465,0,67843,8.236685,6,22,107,5651,622,6107,22107,622107,42,10,18,14,0
164729,2018-10-18 14:46:00,0,268465,0,67843,8.236685,6,22,107,5651,622,6107,22107,622107,42,10,18,14,0
164774,2018-10-18 14:47:00,0,268465,0,67843,8.236685,6,22,107,5651,622,6107,22107,622107,42,10,18,14,0
338978,2018-10-22 09:48:00,0,295495,0,63376,5.808142,14,17,31,6489,1417,1431,1731,141731,43,10,22,9,0
538172,2018-10-26 09:07:00,0,675965,0,82549,6.981006,5,53,79,3993,553,579,5379,55379,43,10,26,9,0


In [24]:
aggs = {}
aggs['item_price'] = ['median', 'mean', 'sum', 'max', 'min','var', q1, q3,  mode, mode_1, mode_2]
aggs['device_type'] = ['count', 'nunique', mode, mode_1, mode_2]
aggs['user_id'] = ['count', 'nunique']
aggs['session_id'] = ['count', 'nunique']
aggs['item_id'] =   ['nunique', mode, mode_1, mode_2]
aggs['category_1'] = ['nunique', mode, mode_1, mode_2]
aggs['category_2'] = ['nunique', mode, mode_1, mode_2]
aggs['category_3'] = ['nunique', mode, mode_1, mode_2]
aggs['cat1_cat2'] = ['nunique', mode, mode_1, mode_2]
aggs['cat1_cat3'] = ['nunique', mode, mode_1, mode_2]
aggs['cat2_cat3'] = ['nunique', mode, mode_1, mode_2]
aggs['cat1_cat2_cat3'] = ['nunique', mode, mode_1, mode_2]
aggs['product_type'] = ['nunique', mode, mode_1, mode_2]

aggs['server_wk'] = ['nunique', mode, mode_1, mode_2, 'mean']
aggs['server_mth'] = ['nunique', mode, mode_1, mode_2, 'mean']
aggs['server_day'] = ['nunique', mode, mode_1, mode_2, 'mean']
aggs['server_hr'] = ['nunique', mode, mode_1, mode_2, 'mean']
aggs['server_weekend'] = ['nunique', mode, mode_1, mode_2, 'sum']

In [25]:
#,'category_1','category_2','category_3'

In [26]:
# for col in tqdm(["server_day", "server_hr",'product_type','cat1_cat2_cat3']):
    
#     aggs_temp = aggs.copy()
#     aggs_temp.pop(col)
    
#     agg_df = df_log_item.groupby(col).agg(aggs_temp).reset_index()
#     agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
#     df_log_item = pd.merge(left = df_log_item, right= agg_df, how='left',
#                     left_on=[col], right_on=[col])

In [27]:
for col in tqdm(["user_id"]):
    
    aggs_temp = aggs.copy()
    aggs_temp.pop(col)
    
    agg_df = df_log_item.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    df_final = pd.merge(left = df_combined, right= agg_df, how='left',
                    left_on=[col], right_on=[col])

100%|██████████| 1/1 [03:13<00:00, 193.03s/it]


In [28]:
#df_final.columns

In [29]:
df_final['impression_time'] = pd.to_datetime(df_final['impression_time'])

df_final['impression_wk'] =  df_final['impression_time'].dt.week
df_final['impression_mth'] = df_final['impression_time'].dt.month
df_final['impression_day'] = df_final['impression_time'].dt.day
df_final['impression_hr'] =  df_final['impression_time'].dt.hour

df_final['impression_weekend'] = (df_final['impression_time'].dt.weekday >= 5).astype(int)

In [30]:
#df_final1 = df_final.copy()

In [31]:
# list_join_cols = ['impression_day','impression_hr','impression_wk','impression_weekend']

# for i,col in tqdm(enumerate(["server_day", "server_hr",'server_wk','server_weekend'])):
    
#     aggs_temp = aggs.copy()
#     aggs_temp.pop(col)
    
#     agg_df = df_log_item.groupby(col).agg(aggs_temp).reset_index()
#     agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
#     df_final1 = pd.merge(left = df_final1, right= agg_df, how='left',
#                     left_on=[list_join_cols[i]], right_on=[col])

In [32]:
#df_final1 = df_final1.drop(["server_day", "server_hr",'server_wk','server_weekend'],axis=1)

In [33]:
df_final['user_time_int1'] = df_final['impression_hr'] + df_final['user_id_item_price_mean']
df_final['user_time_int2'] = df_final['impression_day'] + df_final['user_id_item_price_mean']
df_final['session_cat_int1'] = df_final['user_id_session_id_nunique'] + df_final['user_id_item_price_mean']
df_final['app_price_int'] =  df_final['app_code'] + df_final['user_id_item_price_mean']


In [34]:
c1 = ['app_code', 'os_version'] 

for col in c1:
    le = preprocessing.LabelEncoder()
    df_final[col] = le.fit_transform(df_final[col])

In [ ]:
# df_final['time_app_int1'] = df_final['impression_hr'] + df_final['app_code']
# df_final['time_app_int2'] = df_final['impression_day'] + df_final['app_code']
# df_final['time_os_int1'] = df_final['impression_hr'] + df_final['os_version']
# df_final['time_os_int2'] = df_final['impression_day'] + df_final['os_version']
# df_final['time_g_int1'] = df_final['impression_hr'] + df_final['is_4G']
# df_final['time_g_int2'] = df_final['impression_day'] + df_final['is_4G']
# df_final['g_prc_int'] = df_final['user_id_item_price_mean'] + df_final['is_4G']
# df_final['os_prc_int'] = df_final['user_id_item_price_mean'] + df_final['os_version']

# df_final['session_cat_int1'] = df_final['user_id_session_id_nunique'] + df_final['user_id_category_1_nunique']
# df_final['session_cat_int2'] = df_final['user_id_session_id_nunique'] + df_final['user_id_category_2_nunique']
# df_final['session_cat_int3'] = df_final['user_id_session_id_nunique'] + df_final['user_id_category_3_nunique']


In [ ]:
# df_final['user_time_int1'] = df_final['impression_hr'] + df_final['user_id_item_price_mean']
# df_final['user_time_int2'] = df_final['impression_day'] + df_final['user_id_item_price_mean']
# #df_final['user_time_int3'] = df_final['impression_wk'] + df_final['user_id_item_price_mean']
# #df_final['user_time_int4'] = df_final['impression_weekend'] + df_final['user_id_item_price_mean']

# df_final['session_cat_int1'] = df_final['user_id_session_id_nunique'] + df_final['user_id_item_price_mean']
# df_final['app_price_int'] = df_final['app_code'] + df_final['user_id_item_price_mean']
# #df_final['app_price_int'] = df_final['app_code'] + df_final['user_id_item_price_mean']

Reading Item Vector

In [35]:
df_embed = pd.read_csv('/Users/s0c02nj/Desktop/WNS2/golden_feats/user_embed_20.csv')

In [36]:
col = 'user_id'
df_final1 = pd.merge(left = df_final, right= df_embed, how='left',
                    left_on=[col], right_on=[col])

In [37]:
df_final = df_final.drop(['impression_id', 'impression_time', 'user_id'], axis =1)

In [38]:
df_final1 = df_final1.drop(['impression_id', 'impression_time', 'user_id'], axis =1)

In [39]:
col_counts = []

for col in tqdm(df_final.columns):
    
    counter = Counter(df_final[col])
    col_counts.append(str(col)+'_count')
    
    df_final1[str(col)+'_count'] = df_final1[col].apply(lambda x:counter[x])

100%|██████████| 91/91 [00:32<00:00,  5.08it/s]


In [40]:
df_final = df_final1

In [41]:
cat_appended_cols = []
for col in df_final.columns:
    if df_final[col].dtype == "object":
        cat_appended_cols.append(col)
        print(col , df_final[col].dtype)

user_id_cat1_cat2_mode object
user_id_cat1_cat2_mode_1 object
user_id_cat1_cat2_mode_2 object
user_id_cat1_cat3_mode object
user_id_cat1_cat3_mode_1 object
user_id_cat1_cat3_mode_2 object
user_id_cat2_cat3_mode object
user_id_cat2_cat3_mode_1 object
user_id_cat2_cat3_mode_2 object
user_id_cat1_cat2_cat3_mode object
user_id_cat1_cat2_cat3_mode_1 object
user_id_cat1_cat2_cat3_mode_2 object


In [42]:
cat_cols_train = ['app_code', 'os_version'] + cat_appended_cols

for col in cat_cols_train:
    print(col)
    le = preprocessing.LabelEncoder()
    df_final[col] = le.fit_transform(df_final[col].astype(int))

app_code
os_version
user_id_cat1_cat2_mode
user_id_cat1_cat2_mode_1
user_id_cat1_cat2_mode_2
user_id_cat1_cat3_mode
user_id_cat1_cat3_mode_1
user_id_cat1_cat3_mode_2
user_id_cat2_cat3_mode
user_id_cat2_cat3_mode_1
user_id_cat2_cat3_mode_2
user_id_cat1_cat2_cat3_mode
user_id_cat1_cat2_cat3_mode_1
user_id_cat1_cat2_cat3_mode_2


In [43]:
# for col in tqdm(df_result.columns) :
#     le = preprocessing.LabelEncoder()
#     df_result[col] = le.fit_transform(df_result[col].astype(str))

In [44]:
x_train = df_final[0 : df_train.shape[0]]
x_test = df_final[df_train.shape[0] : ]
y = is_click

In [45]:
x_t, x_v, y_t, y_v = train_test_split(x_train, y, stratify=y, random_state=42, test_size=0.1)

In [46]:
params = {
    'boost': 'gbdt',
    'learning_rate': 0.01,
    'max_depth': 15,  
    'metric':{'auc'},
    'num_threads': -1,
    'objective': 'binary', 
    'verbosity': 1,
    'num_leaves': 30,
    'bagging_fraction':0.7, #rows
    'bagging_frequency':10,
    'min_data_in_leaf':30,
    'feature_fraction':0.2, # features
    'lambda_l1':10,
    'lambda_l2':10,
    'min_gain_split':0.3
}

In [49]:
d_train = lgb.Dataset(x_t, label=y_t)
d_val = lgb.Dataset(x_v, label=y_v)

clf = lgb.train(params, d_train, 150000, verbose_eval=1, 
                valid_sets = [d_train, d_val], early_stopping_rounds = 500)

[1]	training's auc: 0.686096	valid_1's auc: 0.685324
Training until validation scores don't improve for 500 rounds.
[2]	training's auc: 0.700472	valid_1's auc: 0.697686
[3]	training's auc: 0.702891	valid_1's auc: 0.698243
[4]	training's auc: 0.711528	valid_1's auc: 0.705089
[5]	training's auc: 0.716	valid_1's auc: 0.710809
[6]	training's auc: 0.715742	valid_1's auc: 0.709203
[7]	training's auc: 0.717189	valid_1's auc: 0.711976
[8]	training's auc: 0.717703	valid_1's auc: 0.711981
[9]	training's auc: 0.716379	valid_1's auc: 0.710313
[10]	training's auc: 0.719127	valid_1's auc: 0.713908
[11]	training's auc: 0.718253	valid_1's auc: 0.712417
[12]	training's auc: 0.71678	valid_1's auc: 0.711932
[13]	training's auc: 0.717856	valid_1's auc: 0.712907
[14]	training's auc: 0.719947	valid_1's auc: 0.714549
[15]	training's auc: 0.721625	valid_1's auc: 0.716143
[16]	training's auc: 0.723267	valid_1's auc: 0.717868
[17]	training's auc: 0.723763	valid_1's auc: 0.717809
[18]	training's auc: 0.723891	va

[154]	training's auc: 0.746649	valid_1's auc: 0.735039
[155]	training's auc: 0.746749	valid_1's auc: 0.735085
[156]	training's auc: 0.746913	valid_1's auc: 0.73515
[157]	training's auc: 0.747128	valid_1's auc: 0.735367
[158]	training's auc: 0.747179	valid_1's auc: 0.735397
[159]	training's auc: 0.747285	valid_1's auc: 0.73546
[160]	training's auc: 0.747352	valid_1's auc: 0.73539
[161]	training's auc: 0.747476	valid_1's auc: 0.73552
[162]	training's auc: 0.747571	valid_1's auc: 0.735541
[163]	training's auc: 0.747682	valid_1's auc: 0.735573
[164]	training's auc: 0.747934	valid_1's auc: 0.735776
[165]	training's auc: 0.748043	valid_1's auc: 0.735784
[166]	training's auc: 0.748274	valid_1's auc: 0.735955
[167]	training's auc: 0.74848	valid_1's auc: 0.736144
[168]	training's auc: 0.748565	valid_1's auc: 0.736195
[169]	training's auc: 0.748842	valid_1's auc: 0.736446
[170]	training's auc: 0.749006	valid_1's auc: 0.736615
[171]	training's auc: 0.749087	valid_1's auc: 0.736632
[172]	training'

[306]	training's auc: 0.762558	valid_1's auc: 0.744924
[307]	training's auc: 0.762652	valid_1's auc: 0.744966
[308]	training's auc: 0.762722	valid_1's auc: 0.74497
[309]	training's auc: 0.762788	valid_1's auc: 0.744978
[310]	training's auc: 0.76293	valid_1's auc: 0.745136
[311]	training's auc: 0.76303	valid_1's auc: 0.745191
[312]	training's auc: 0.763129	valid_1's auc: 0.745207
[313]	training's auc: 0.763193	valid_1's auc: 0.745272
[314]	training's auc: 0.763225	valid_1's auc: 0.745302
[315]	training's auc: 0.763339	valid_1's auc: 0.745361
[316]	training's auc: 0.763415	valid_1's auc: 0.745344
[317]	training's auc: 0.763502	valid_1's auc: 0.745329
[318]	training's auc: 0.763592	valid_1's auc: 0.745468
[319]	training's auc: 0.763677	valid_1's auc: 0.745474
[320]	training's auc: 0.763798	valid_1's auc: 0.745529
[321]	training's auc: 0.7639	valid_1's auc: 0.745609
[322]	training's auc: 0.763974	valid_1's auc: 0.745652
[323]	training's auc: 0.76407	valid_1's auc: 0.745776
[324]	training's

[458]	training's auc: 0.774546	valid_1's auc: 0.750437
[459]	training's auc: 0.77461	valid_1's auc: 0.750447
[460]	training's auc: 0.774686	valid_1's auc: 0.750484
[461]	training's auc: 0.77478	valid_1's auc: 0.750492
[462]	training's auc: 0.774866	valid_1's auc: 0.750536
[463]	training's auc: 0.774916	valid_1's auc: 0.750558
[464]	training's auc: 0.774992	valid_1's auc: 0.750525
[465]	training's auc: 0.775066	valid_1's auc: 0.750573
[466]	training's auc: 0.775171	valid_1's auc: 0.75065
[467]	training's auc: 0.775248	valid_1's auc: 0.750685
[468]	training's auc: 0.775298	valid_1's auc: 0.750694
[469]	training's auc: 0.775377	valid_1's auc: 0.750705
[470]	training's auc: 0.77544	valid_1's auc: 0.750732
[471]	training's auc: 0.775523	valid_1's auc: 0.750768
[472]	training's auc: 0.775621	valid_1's auc: 0.750813
[473]	training's auc: 0.775671	valid_1's auc: 0.750841
[474]	training's auc: 0.775722	valid_1's auc: 0.75083
[475]	training's auc: 0.775794	valid_1's auc: 0.750844
[476]	training'

[612]	training's auc: 0.785273	valid_1's auc: 0.754622
[613]	training's auc: 0.785346	valid_1's auc: 0.754661
[614]	training's auc: 0.785386	valid_1's auc: 0.754659
[615]	training's auc: 0.78544	valid_1's auc: 0.754672
[616]	training's auc: 0.785493	valid_1's auc: 0.754704
[617]	training's auc: 0.785548	valid_1's auc: 0.754692
[618]	training's auc: 0.785608	valid_1's auc: 0.754707
[619]	training's auc: 0.785669	valid_1's auc: 0.754726
[620]	training's auc: 0.785719	valid_1's auc: 0.754766
[621]	training's auc: 0.785785	valid_1's auc: 0.754782
[622]	training's auc: 0.785827	valid_1's auc: 0.754742
[623]	training's auc: 0.785881	valid_1's auc: 0.754764
[624]	training's auc: 0.78594	valid_1's auc: 0.754819
[625]	training's auc: 0.786005	valid_1's auc: 0.754828
[626]	training's auc: 0.786069	valid_1's auc: 0.75487
[627]	training's auc: 0.786124	valid_1's auc: 0.754895
[628]	training's auc: 0.786183	valid_1's auc: 0.754904
[629]	training's auc: 0.786257	valid_1's auc: 0.75493
[630]	training

...

[5416]	training's auc: 0.907914	valid_1's auc: 0.772206
[5417]	training's auc: 0.907924	valid_1's auc: 0.77221
[5418]	training's auc: 0.907937	valid_1's auc: 0.772208
[5419]	training's auc: 0.907952	valid_1's auc: 0.772209
[5420]	training's auc: 0.907966	valid_1's auc: 0.77222
[5421]	training's auc: 0.907984	valid_1's auc: 0.772217
[5422]	training's auc: 0.907998	valid_1's auc: 0.772204
[5423]	training's auc: 0.908006	valid_1's auc: 0.772205
[5424]	training's auc: 0.908013	valid_1's auc: 0.772204
[5425]	training's auc: 0.908025	valid_1's auc: 0.77219
[5426]	training's auc: 0.908041	valid_1's auc: 0.772189
[5427]	training's auc: 0.908054	valid_1's auc: 0.772181
[5428]	training's auc: 0.908068	valid_1's auc: 0.772188
[5429]	training's auc: 0.908082	valid_1's auc: 0.772186
[5430]	training's auc: 0.908098	valid_1's auc: 0.772187
[5431]	training's auc: 0.908109	valid_1's auc: 0.772185
[5432]	training's auc: 0.908123	valid_1's auc: 0.772182
[5433]	training's auc: 0.90814	valid_1's auc: 0.772

[5565]	training's auc: 0.909743	valid_1's auc: 0.772397
[5566]	training's auc: 0.909752	valid_1's auc: 0.772402
[5567]	training's auc: 0.909759	valid_1's auc: 0.772398
[5568]	training's auc: 0.909772	valid_1's auc: 0.772397
[5569]	training's auc: 0.909786	valid_1's auc: 0.772408
[5570]	training's auc: 0.909801	valid_1's auc: 0.77241
[5571]	training's auc: 0.909814	valid_1's auc: 0.772417
[5572]	training's auc: 0.909831	valid_1's auc: 0.772431
[5573]	training's auc: 0.909842	valid_1's auc: 0.772431
[5574]	training's auc: 0.90985	valid_1's auc: 0.772435
[5575]	training's auc: 0.90986	valid_1's auc: 0.772435
[5576]	training's auc: 0.909873	valid_1's auc: 0.772425
[5577]	training's auc: 0.909891	valid_1's auc: 0.772417
[5578]	training's auc: 0.909906	valid_1's auc: 0.772421
[5579]	training's auc: 0.909918	valid_1's auc: 0.772424
[5580]	training's auc: 0.909926	valid_1's auc: 0.772421
[5581]	training's auc: 0.90994	valid_1's auc: 0.772429
[5582]	training's auc: 0.909951	valid_1's auc: 0.772

[5716]	training's auc: 0.911534	valid_1's auc: 0.772425
[5717]	training's auc: 0.911547	valid_1's auc: 0.772419
[5718]	training's auc: 0.911559	valid_1's auc: 0.772412
[5719]	training's auc: 0.911571	valid_1's auc: 0.772411
[5720]	training's auc: 0.911576	valid_1's auc: 0.772418
[5721]	training's auc: 0.911587	valid_1's auc: 0.772408
[5722]	training's auc: 0.911597	valid_1's auc: 0.772395
[5723]	training's auc: 0.911606	valid_1's auc: 0.772384
[5724]	training's auc: 0.911615	valid_1's auc: 0.772377
[5725]	training's auc: 0.911625	valid_1's auc: 0.772383
[5726]	training's auc: 0.911639	valid_1's auc: 0.772366
[5727]	training's auc: 0.911651	valid_1's auc: 0.772368
[5728]	training's auc: 0.911658	valid_1's auc: 0.772365
[5729]	training's auc: 0.911668	valid_1's auc: 0.772363
[5730]	training's auc: 0.911676	valid_1's auc: 0.772356
[5731]	training's auc: 0.911693	valid_1's auc: 0.772366
[5732]	training's auc: 0.911706	valid_1's auc: 0.77237
[5733]	training's auc: 0.91172	valid_1's auc: 0.7

[5866]	training's auc: 0.913257	valid_1's auc: 0.772164
[5867]	training's auc: 0.913265	valid_1's auc: 0.77217
[5868]	training's auc: 0.913278	valid_1's auc: 0.772167
[5869]	training's auc: 0.913287	valid_1's auc: 0.772168
[5870]	training's auc: 0.913293	valid_1's auc: 0.772178
[5871]	training's auc: 0.913302	valid_1's auc: 0.77218
[5872]	training's auc: 0.913316	valid_1's auc: 0.772182
[5873]	training's auc: 0.91333	valid_1's auc: 0.772167
[5874]	training's auc: 0.91334	valid_1's auc: 0.772161
[5875]	training's auc: 0.913352	valid_1's auc: 0.772164
[5876]	training's auc: 0.91336	valid_1's auc: 0.772167
[5877]	training's auc: 0.913373	valid_1's auc: 0.772163
[5878]	training's auc: 0.913384	valid_1's auc: 0.77217
[5879]	training's auc: 0.913395	valid_1's auc: 0.772158
[5880]	training's auc: 0.913407	valid_1's auc: 0.772156
[5881]	training's auc: 0.913416	valid_1's auc: 0.772147
[5882]	training's auc: 0.913425	valid_1's auc: 0.772162
[5883]	training's auc: 0.91344	valid_1's auc: 0.772158

[6018]	training's auc: 0.914962	valid_1's auc: 0.772126
[6019]	training's auc: 0.91497	valid_1's auc: 0.772118
[6020]	training's auc: 0.91498	valid_1's auc: 0.77212
[6021]	training's auc: 0.91499	valid_1's auc: 0.772122
[6022]	training's auc: 0.915003	valid_1's auc: 0.772116
[6023]	training's auc: 0.915024	valid_1's auc: 0.772128
[6024]	training's auc: 0.915036	valid_1's auc: 0.772131
[6025]	training's auc: 0.915043	valid_1's auc: 0.772122
[6026]	training's auc: 0.915052	valid_1's auc: 0.772115
[6027]	training's auc: 0.915063	valid_1's auc: 0.772108
[6028]	training's auc: 0.915074	valid_1's auc: 0.772116
[6029]	training's auc: 0.915091	valid_1's auc: 0.772115
[6030]	training's auc: 0.9151	valid_1's auc: 0.772127
[6031]	training's auc: 0.915117	valid_1's auc: 0.772131
[6032]	training's auc: 0.915121	valid_1's auc: 0.772131
[6033]	training's auc: 0.915133	valid_1's auc: 0.772133
[6034]	training's auc: 0.915141	valid_1's auc: 0.772125
[6035]	training's auc: 0.915153	valid_1's auc: 0.77210

In [ ]:
#[6382]	training's auc: 0.927222	valid_1's auc: 0.759128
#[6299]	training's auc: 0.931325	valid_1's auc: 0.760466

In [50]:
predictions = clf.predict(x_test)

In [51]:
sub_df = pd.read_csv('/Users/s0c02nj/Desktop/WNS2/sample_submission.csv')
sub_df['is_click'] = predictions
sub_df.to_csv('/Users/s0c02nj/Desktop/WNS2/output_submission/last.csv', index = False)

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importance(), 
                                      x_train.columns), reverse=True), columns=['Value','Feature'])

In [ ]:
#feature_imp

In [ ]:
new_feat_imp = feature_imp[feature_imp['Value'] >84]

In [ ]:
list_new_feat = list(new_feat_imp['Feature'])

In [ ]:
def get_inv_sig(x):
    y = np.log(x/(1-x))
    return y

In [ ]:
def get_sig(x):
    y = 1/(1+np.exp(-x))
    return y

In [ ]:
#Stratified K-fold

In [52]:
kf = StratifiedKFold(n_splits=20, random_state=2019, shuffle=True)

In [53]:
predictions = np.zeros(len(x_test))
num_models = 0

for train_index, valid_index in tqdm(kf.split(x_train, y)):
    
    d_train = lgb.Dataset(x_train.iloc[train_index], label=y[train_index])
    d_val = lgb.Dataset(x_train.iloc[valid_index], label=y[valid_index])

    clf = lgb.train(params, d_train, 150000, verbose_eval=1000, 
                    valid_sets = [d_train, d_val], early_stopping_rounds = 500)
    

    
    num_models += 1
    
    predictions += clf.predict(x_test)

0it [00:00, ?it/s]

Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.805555	valid_1's auc: 0.746926
[2000]	training's auc: 0.84207	valid_1's auc: 0.753274
[3000]	training's auc: 0.866254	valid_1's auc: 0.755682
[4000]	training's auc: 0.884791	valid_1's auc: 0.756685
[5000]	training's auc: 0.89928	valid_1's auc: 0.757125
Early stopping, best iteration is:
[4868]	training's auc: 0.897483	valid_1's auc: 0.757276


1it [02:21, 141.09s/it]

Training until validation scores don't improve for 500 rounds.


KeyboardInterrupt: 

In [ ]:
#predictions_15_new = predictions / 15.0
predictions = predictions/15.0

In [ ]:
sub_df = pd.read_csv('/Users/s0c02nj/Desktop/WNS2/sample_submission.csv')
sub_df['is_click'] = predictions
sub_df.to_csv('/Users/s0c02nj/Desktop/WNS2/output_submission/k_fold_15_new.csv', index = False)

In [ ]:
sub_df = pd.read_csv('sample_submission.csv')
sub_df['is_click'] = pred2
sub_df.to_csv('sub_agg_time_and_timeagg_count_top3_8654_75389_lr01_10_5_div2fold.csv', index = False)

In [ ]:
temp = df_log_item[df_log_item.user_id == 4557].groupby(['user_id', 'session_id'])['item_id'].apply(list)

In [ ]:
a = []
for user_id in tqdm(list(df_log_item["user_id"].unique())):
    temp = df_log_item[df_log_item.user_id == user_id].groupby(['user_id', 'session_id'])['item_id'].apply(list)
    a += list(temp)

In [ ]:
session_length = [len(t) for t in tqdm(a)]

In [ ]:
np.percentile(session_length, 75)

In [ ]:
Counter(session_length)

In [ ]:
df_log_item.columns

In [ ]:
df_feats = pd.merge(left = df_train, 
                  right= df_log_item, 
                  how='left',
                  left_on=['user_id'], 
                  right_on=['user_id'])

In [ ]:
df_log_item.isna().sum()

In [ ]:
df_test_feats = pd.merge(left = df_test, 
                  right= df_log_item, 
                  how='left',
                  left_on=['user_id'], 
                  right_on=['user_id'])

In [ ]:
df_feats.columns

In [ ]:
categorical_features = ["app_code", "os_version"] + ['device_type', 'session_id', 'category_1', 'category_2', 'category_3', 'product_type', 'cat1_cat2', 'cat1_cat3', 'cat2_cat3', 'cat1_cat2_cat3']

In [ ]:
Counter(predictions)

In [ ]:
df_test["is_click"] = predictions

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importance(), 
                                      x_train.columns)), columns=['Value','Feature'])

In [ ]:
feature_imp

In [ ]:
df_test.head()

In [ ]:
sub = df_test[["impression_id", "is_click"]]
sub.to_csv("no_extra_feats_1.csv", index=False)

In [ ]:
df_log_item["category_1"].astype("str") + df_log_item["category_2"].astype("str")

In [ ]:
def hyper_params(md,n_l,bf,bf1,mdl,ff,l1,l2,mgs):
    
    params = {
        'boost': 'gbdt',
        'learning_rate': 0.03,
        'max_depth': md,
        'metric':{'auc'},
        'num_threads': -1,
        'objective': 'binary', 
        'verbosity': 1,
        'num_leaves': n_l,
        'bagging_fraction':bf, #rows
        'bagging_frequency':bf1,
        'min_data_in_leaf':mdl,
        'feature_fraction':ff, # features
        'lambda_l1':l1,
        'lambda_l2':l2,
        'min_gain_split':mgs,
        'is_unbalance':True}
    
    return params

In [ ]:
md_range = np.arange(6,30,1)


In [ ]:
params = {
    'boost': 'gbdt',
    'learning_rate': 0.03,
    'max_depth': 8,  
    'metric':{'auc'},
    'num_threads': -1,
    'objective': 'binary', 
    'verbosity': 1,
    'num_leaves': 8,
    'bagging_fraction':0.5, #rows
    'bagging_frequency':10,
    'min_data_in_leaf':3,
    'feature_fraction':0.35, # features
    'lambda_l1':5,
    'lambda_l2':10,
    'min_gain_split':0.3,
    'is_unbalance':True
}

In [ ]:
a = ['/Users/s0c02nj/Desktop/WNS2/output_submission/fold_user_embed40d_1.csv'
     ,'/Users/s0c02nj/Desktop/WNS2/output_submission/fold_user_embed40d.csv'
    ,'/Users/s0c02nj/Desktop/WNS2/output_submission/fold_user_embed_withot_is_unbalance1.csv'
    ,'/Users/s0c02nj/Desktop/WNS2/output_submission/fold_user_embed_withot_is_unbalance.csv'
    ,'/Users/s0c02nj/Desktop/WNS2/output_submission/fold_user_embed_corrected2.csv'
    ,'/Users/s0c02nj/Desktop/WNS2/output_submission/fold_user_embed_corrected1.csv'
    ,'/Users/s0c02nj/Desktop/WNS2/output_submission/fold_user_embed_corrected.csv'
    ,'/Users/s0c02nj/Desktop/WNS2/output_submission/fold_user_embed_1.csv'
    ,'/Users/s0c02nj/Desktop/WNS2/output_submission/fold_user_embed.csv']

In [ ]:
p1 = np.zeros((df_test.shape[0]))
for i in a :
    p = pd.read_csv(i)
    pred = p['is_click'].values
    p1 = p1+pred

In [ ]:
predf = p1/len(a)

In [ ]:
sub_df = pd.read_csv('/Users/s0c02nj/Desktop/WNS2/sample_submission.csv')
sub_df['is_click'] = predf
sub_df.to_csv('/Users/s0c02nj/Desktop/WNS2/output_submission/ensemble_good.csv', index = False)

In [54]:
a = [1,2,3]

In [56]:
np.save('abcd',a)

In [60]:
np.load('abcd.npy')[0]

1